<a href="https://colab.research.google.com/github/aidowu1/Generative-AI-Recipes/blob/main/Demo_Use_Of_GPT4All.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Demo Use Of GPT4All

There has been a growing interest in Generative AI, especially Large Language Models (LLM) such as GPT-4 and its impact in the development of business solutions and consumer products. Recently, there has been an accessable addition to the LLM ecosystem known as GPT4All.

GPT4All is an offline alternative to GPT4 alternative that can run on your computer using its CPU. GPT4All is developed by [Nomic AI](https://github.com/nomic-ai/gpt4all).

A typical GPT4All model is a 3GB - 8GB file that you can download and run on your local machine. Nomic AI supports and maintains this software ecosystem to enforce quality and security alongside spearheading the effort to allow any person or enterprise to easily train and deploy their own on-edge large language models.

Nomic's GPT4All model offering is currently compatible with three variants of the Transformer neural network architecture, namely:
 - LLaMa
 - GPT-J
 - MPT

Full details of these variants are detailed in this [json file](https://raw.githubusercontent.com/nomic-ai/gpt4all/main/gpt4all-chat/metadata/models.json)

This notebook will provde a quick guide on how to setup and use GPT4ALL to create a really simply Q & A chatbot on a local computer CPU. From my experience, typically to run GPT4All on a local computer you might need at least 16 GB of RAM.This notebook will describe the following steps:
 - Installation of GPT4ALL
 - Install Jupyter Dash to create simple interactive UI controls to demo Chatbot
 - Import the packgages
 - Using GPT4ALL to create a simple Q & A chatbot


### Installation of GPT4All
 - I will be running this on a Google colab instance
 - If you want to install this on you local PC, I will advise you to setup a virtual environment (with python 3.9) using Conda

In [1]:
!pip install gpt4all

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 32.4 MB/s eta 0:00:00


### Install Jupyter Dash to create simple interactive UI controls to demo Chatbot

In [2]:
!pip install -q jupyter-dash pip install "dash-bootstrap-components<1"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.3/197.3 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 120.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 85.1 MB/s eta 0:00:00


### Define imports

In [3]:
import gpt4all
import time
import dash
import dash_html_components as html
import dash_core_components as dcc
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output, State
from jupyter_dash import JupyterDash
import pandas as pd
import json
from enum import Enum, auto

<ipython-input-3-bebd47436a2e>:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
<ipython-input-3-bebd47436a2e>:5: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


### Global constants and variables

In [4]:
# Chat bot model global variable
chatbot = None

# Chat delimiter
DELIMITER = ":"

# Line divider for console output
LINE_DIVIDER = "==========" * 5


class gpt4all_model_type(Enum):
  gpt_j = auto()
  llama = auto()
  mpt = auto()

GPT4ALL_MODELS = {
    gpt4all_model_type.gpt_j: "ggml-gpt4all-j-v1.3-groovy.bin",
    gpt4all_model_type.llama: "GPT4All-13B-snoozy.ggmlv3.q4_0.bin",
    gpt4all_model_type.mpt: "ggml-mpt-7b-base.bin",
}

### Define the Chat bot based on GPT4All Q & A inferencing

In [5]:
class GPT4AllChatBot(object):
  """
  GPT4All Chat Bot
  """
  def __init__(self, model_type: gpt4all_model_type=gpt4all_model_type.gpt_j):
    """
    Constructor
    :param model_type: Model type
    """
    print("This is a simple Chat Bot using GPT4All inferencing...")
    self._model_type = GPT4ALL_MODELS[model_type]
    self._model = gpt4all.GPT4All(self._model_type)

  def _run(self, question: str) -> str:
    """
    Runs the Q & A inferencing
    :param question: Prompt/question
    :return: Response/answer
    """
    try:
      request = [{"role": "user", "content": question}]
      answer = self._model.generate(question, streaming=False)
      response = json.dumps(answer, indent=4)
    except Exception as ex:
      error_msg = f"{str(ex) } Error has occurred\n"
      raise Exception(error_msg)
    return response

  def computeChatResponse(self, prompt: str) -> str:
    """
    Computes the chatbot response
    :param question: Prompt/question
    :return: Response/answer
    """
    response = self._run(prompt).replace('"', '').replace('\\n', '')
    response = f"{prompt}: {response}"
    return response



### Instantiate the Chat Bot NLP Model

In [6]:
if chatbot == None:
  chatbot = GPT4AllChatBot(model_type=gpt4all_model_type.gpt_j)

This is a simple Chat Bot using GPT4All inferencing...


100%|██████████| 3.79G/3.79G [01:25<00:00, 44.2MiB/s]


Model downloaded at:  /root/.cache/gpt4all/ggml-gpt4all-j-v1.3-groovy.bin


### Test it on a simple Q & A task:

In [7]:
def testQAndAInferencing():
  """
  Invokes the Q and A inferencing tests
  """
  LINE_DIVIDER = "==========" * 5
  prompts = [
      "What is 12 + 13?",
      "What is the square root of 144?",
      "What is reinforcement Learning?",
      "What is semi-supervised Learning?",
      "Who was the president US from 2008 to 2012?",
      "Which team won the English premier league in 2012?",
      ]
  for prompt in prompts:
    start_time = time.time()
    response = chatbot.computeChatResponse(prompt)
    elasped_time = round(time.time() - start_time, 2)
    print(f"Question: {prompt}\tAnswer:{response}")
    print(f"Inference time: {elasped_time}")
    print(f"{LINE_DIVIDER}\n\n")

testQAndAInferencing()


Question: What is 12 + 13?	Answer:What is 12 + 13?: 12 plus 13 equals 25.
Inference time: 17.65



Question: What is the square root of 144?	Answer:What is the square root of 144?: The square root of 144 is 12.
Inference time: 7.22



Question: What is reinforcement Learning?	Answer:What is reinforcement Learning?: Reinforcement learning (RL) is a type of machine learning algorithm that involves training an agent to make decisions in a complex environment. The goal is to maximize the cumulative reward over time, which can be achieved by learning from the environment and making decisions based on that experience. The agent receives a reward for each action it takes, and the environment provides feedback in the form of rewards or penalties. The agent then adjusts its behavior based on the feedback to maximize cumulative reward over time.
Inference time: 48.97



Question: What is semi-supervised Learning?	Answer:What is semi-supervised Learning?: Semi-supervised learning is a type of ma

### Create interactive UI controls for the Chat Bot

#### Create chat bot text box control

In [8]:
DELIMITER = ":"

def textbox(text, box="other"):
    style = {
        "max-width": "55%",
        "width": "max-content",
        "padding": "10px 15px",
        "border-radius": "25px",
    }

    if box == "self":
        style["margin-left"] = "auto"
        style["margin-right"] = 0

        color = "primary"
        inverse = True

    elif box == "other":
        style["margin-left"] = 0
        style["margin-right"] = "auto"

        color = "light"
        inverse = False

    else:
        raise ValueError("Incorrect option for `box`.")

    return dbc.Card(text, style=style, body=True, color=color, inverse=inverse)


#### Create the chat bot UI layout

In [9]:
conversation = html.Div(
    style={
        "width": "80%",
        "max-width": "800px",
        "height": "70vh",
        "margin": "auto",
        "overflow-y": "auto",
    },
    id="display-conversation",
)

controls = dbc.InputGroup(
    style={"width": "80%", "max-width": "800px", "margin": "auto"},
    children=[
        dbc.Input(id="user-input", placeholder="Write to the chatbot...", type="text"),
        dbc.InputGroupAddon(dbc.Button("Submit", id="submit"), addon_type="append",),
    ],
)

# Define app
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
server = app.server


# Define Layout
app.layout = dbc.Container(
    fluid=True,
    children=[
        html.H1("Demo of GPT4All Chatbot"),
        html.Hr(),
        dcc.Store(id="store-conversation", data=""),
        conversation,
        controls,
    ],
)


#### Define chat bot UI callback

In [10]:
@app.callback(
    Output("display-conversation", "children"), [Input("store-conversation", "data")]
)
def updateDisplay(chat_history):
    messages = [
        textbox(x, box="self") if i % 2 == 0 else textbox(x, box="other")
        for i, x in enumerate(chat_history.split(DELIMITER))
    ]
    return messages


@app.callback(
    [Output("store-conversation", "data"), Output("user-input", "value")],
    [Input("submit", "n_clicks"), Input("user-input", "n_submit")],
    [State("user-input", "value"), State("store-conversation", "data")],
)
def runChatbot(n_clicks, n_submit, user_input, chat_history):
    if n_clicks == 0:
        return "", ""

    if user_input is None or user_input == "":
        return chat_history, ""
    response = chatbot.computeChatResponse(user_input)
    chat_history = chat_history + ":" + response
    return chat_history, ""


### Run the chat Bot

In [11]:
app.run_server(mode='inline', port="8099")

Dash is running on http://127.0.0.1:8099/



INFO:dash.dash:Dash is running on http://127.0.0.1:8099/



<IPython.core.display.Javascript object>